# Earthquakes API example
This example shows how to make a very simple API call and pull the data into a Pandas data frame.

We will extract a list of the latest earthquakes using an API from the US Geological Survey site.  See here for more details:

https://earthquake.usgs.gov/


In [1]:
# Import the libraries
import requests
import json
import pandas as pd

This function just returns the URL of the API:

In [2]:
# Function to put together the API query
def getRequestUrl():
    url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_hour.geojson"
    return url

This function calls the API and returns the data of interest as a JSON object:

In [3]:
# Function to call the API with the given query URL
def getApiData(requestUrl):
    # Call the api
    response = requests.get(requestUrl)
    
    # Extract the text from the response
    data = json.loads(response.text)
    
    # Get the actual data of interest (as specfied in the earthquakes api)
    results = data["features"]
    
    # Get other information (as specfied in the earthquakes api)
    count = data["metadata"]["count"]
    print("Response: count {}".format(count))
    
    return results


Now we can extract the data and put it into a Pandas dataframe:

In [4]:
# Get the data
requestUrl = getRequestUrl()
results = getApiData(requestUrl)
df = pd.DataFrame(results)
df.head()

Response: count 11


,type,properties,geometry,id
0,Feature,"{'mag': 0.69, 'place': '20km E of Little Lake,...","{'type': 'Point', 'coordinates': [-117.6815, 3...",ci39712599
1,Feature,"{'mag': 0.92, 'place': '14km ENE of Ridgecrest...","{'type': 'Point', 'coordinates': [-117.5226667...",ci39712591
2,Feature,"{'mag': 1.4, 'place': '46 km W of Anchor Point...","{'type': 'Point', 'coordinates': [-152.6595, 5...",ak0217tbbun3
3,Feature,"{'mag': 1.28, 'place': '14km WSW of Niland, CA...","{'type': 'Point', 'coordinates': [-115.6593333...",ci39712583
4,Feature,"{'mag': 1.66, 'place': '16km WNW of Progreso, ...","{'type': 'Point', 'coordinates': [-115.7401667...",ci39712567


Because we have nested json, it might be better to normalize this:

In [5]:
from pandas.io.json import json_normalize

In [6]:
df = json_normalize(results)
df.head()

,type,id,properties.mag,properties.place,properties.time,properties.updated,properties.tz,properties.url,properties.detail,properties.felt,...,properties.types,properties.nst,properties.dmin,properties.rms,properties.gap,properties.magType,properties.type,properties.title,geometry.type,geometry.coordinates
0,Feature,ci39712599,0.69,"20km E of Little Lake, CA",1624097153050,1624097366295,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,None,...,",nearby-cities,origin,phase-data,scitech-link,",8.0,0.02974,0.07,88.0,ml,earthquake,"M 0.7 - 20km E of Little Lake, CA",Point,"[-117.6815, 35.9321667, 4.17]"
1,Feature,ci39712591,0.92,"14km ENE of Ridgecrest, CA",1624097033850,1624097245676,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,None,...,",nearby-cities,origin,phase-data,scitech-link,",14.0,0.08794,0.20,136.0,ml,earthquake,"M 0.9 - 14km ENE of Ridgecrest, CA",Point,"[-117.5226667, 35.6595, 1.23]"
2,Feature,ak0217tbbun3,1.40,"46 km W of Anchor Point, Alaska",1624096961598,1624097591271,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,None,...,",origin,",NaN,NaN,0.34,NaN,ml,earthquake,"M 1.4 - 46 km W of Anchor Point, Alaska",Point,"[-152.6595, 59.7523, 87.9]"
3,Feature,ci39712583,1.28,"14km WSW of Niland, CA",1624096137680,1624096354583,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,None,...,",nearby-cities,origin,phase-data,scitech-link,",20.0,0.03864,0.28,95.0,ml,earthquake,"M 1.3 - 14km WSW of Niland, CA",Point,"[-115.6593333, 33.2095, 2.74]"
4,Feature,ci39712567,1.66,"16km WNW of Progreso, B.C., MX",1624095946280,1624096165027,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/earthquakes/feed/v...,None,...,",nearby-cities,origin,phase-data,scitech-link,",14.0,0.01163,0.20,160.0,ml,earthquake,"M 1.7 - 16km WNW of Progreso, B.C., MX",Point,"[-115.7401667, 32.6493333, 5.84]"
